In [4]:
import csv
import torch
import numpy as np
import pandas as pd
import copy
import networkx as nx
import random
import matplotlib.pyplot as plt
from math import sqrt 


In [5]:
# Load the CSV file into a DataFrame
data = pd.read_csv("train.csv")

# Display the first five rows of the DataFrame
print(data.shape)
data.head()

(1458644, 11)


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [11]:
# Convert the pickup and dropoff timestamps to datetime objects
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])

In [24]:
data2_5=copy.deepcopy(data)
data2_5['pickup_longitude'] = data2_5['pickup_longitude'].apply(lambda x: round(x / 0.005) * 0.005)
data2_5['pickup_latitude'] = data2_5['pickup_latitude'].apply(lambda x: round(x / 0.005) * 0.005)
data2_5['dropoff_longitude'] = data2_5['dropoff_longitude'].apply(lambda x: round(x / 0.005) * 0.005)
data2_5['dropoff_latitude'] = data2_5['dropoff_latitude'].apply(lambda x: round(x / 0.005) *0.005)


###
repeat 


In [25]:
# creating data for 1.5 hours : 
dataOneHour2=copy.deepcopy(data2_5)
dataOneHour2 = dataOneHour2[(dataOneHour2['pickup_datetime'] >= '2016-01-04 08:00:00') & (dataOneHour2['pickup_datetime'] <= '2016-01-04 09:30:00')]

In [26]:
# Create a directed graph
G25 = nx.DiGraph()
selfloops = 0
# Add edges to the graph
for row in dataOneHour2.itertuples():
    source = (row.pickup_longitude, row.pickup_latitude)
    destination = (row.dropoff_longitude, row.dropoff_latitude)
    weight = row.trip_duration
    if (source != destination):
      G25.add_edge(source, destination, weight=weight)
    else :
      selfloops+=1

print(f'num of self loops in data2 = {selfloops}')

num of self loops in data2 = 6


In [27]:
num_edges = G25.number_of_edges()
print(f"Number of edges in the graph: {num_edges}")

num_nodes = G25.number_of_nodes()
print(f"Number of nodes in the graph: {num_nodes}")

Number of edges in the graph: 544
Number of nodes in the graph: 212


In [28]:
dataOneHour2.to_csv('data_8_930.csv', index=False)


OSError: Cannot save file into a non-existent directory: 'dataset/traffic/spatial'

In [ ]:
def calc_air_distance(lat1, lon1, lat2, lon2):
    # Assuming 1 degree of latitude and longitude is approximately 111 kilometers
    lat_km = 111.0
    lon_km = 90.0

    # Calculate the difference in coordinates
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1

    # Calculate the distance using Euclidean distance formula
    distance = sqrt((delta_lat * lat_km)**2 + (delta_lon * lon_km)**2)
    return distance

In [ ]:
# Create an empty features matrix
features_matrix = np.zeros((num_nodes, 4))

# Loop through each node
for i, node in enumerate(G.nodes()):
    # Get the node attributes
    longitude = node[0]
    latitude = node[1]
    
    # Calculate the average weight of all edges from the node
    traffic_from_avg = np.mean([edge[2]['weight'] for edge in G.out_edges(node, data=True)])
    
    # Calculate the average weight of all edges into the node
    traffic_into_avg = np.mean([edge[2]['weight'] for edge in G.in_edges(node, data=True)])
    
    # Assign the features to the features matrix
    features_matrix[i] = [longitude, latitude, traffic_from_avg, traffic_into_avg]  # Add the two additional features here
# Print the features matrix
print(features_matrix)


In [ ]:
# A = nx.adjacency_matrix(G, nodelist = sorted(list(sorted_labels.keys())), weight = 'weight')
A = nx.adjacency_matrix(G,  weight = 'weight')
A.shape

(212, 212)

In [ ]:
# Convert the matrix A to a DataFrame
df_A = pd.DataFrame.sparse.from_spmatrix(A)

# Save the DataFrame as a CSV file
df_A.to_csv('dataset/traffic/spatial/A_matrix.csv', index=True)


In [ ]:
# path = 'dataset/spatial'
nx.write_edgelist(G,'dataset/traffic/spatial/traffic.edges', delimiter=',', data=[('weight', float)])


In [ ]:
# Save the attributes to the npz file
np.savez('dataset/traffic/spatial/traffic.npz', adj=A, features=features_matrix)


NameError: name 'new_labels' is not defined

In [ ]:
# Load the npz file
data = np.load('dataset/traffic/spatial/traffic.npz')

# Check if features_matrix has a None value
if np.any(data['features'] is None):
    print("features_matrix has a None value")
else:
    print("features_matrix does not have a None value")


features_matrix does not have a None value
